# Generate a synthetic dataset

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

# Constants for reproducibility
SEED = 42
np.random.seed(SEED)


X = np.linspace(-0.5, 0.5, 150).astype('float64')
noise = np.random.normal(0, 0.1, len(X)).astype('float64')
y = 2 * X + 5 + noise # y=2x+5 + error

print('X values = ',X)
print('y values = ', y)
   


In [ ]:
plt.scatter(X,y)
plt.xlabel('X values')
plt.ylabel('y values')

# Equation of a straight line 

<br>

<img src="line_equation.gif" width="200px" height="200px" align="left">   <br><br>

**y:** y values  
**x:** x values  
**b:** gradient  
**a:** y intercept  


# Calculating the gradient of the line of best fit
<br>

<img src="gradient.png" width="230px" height="230px" align="left">

In [ ]:
numerator = sum((X - X.mean()) * (y - y.mean()))
denominator = sum((X - X.mean())**2)

beta = numerator / denominator
print('The slope of regression line:', beta)

# Calculating the y intercept

<br>

<img src="intercept.png" width="150px" height="150px" align="left">

In [ ]:
intercept = y.mean() - beta * X.mean()
print('The y intercept of our regression line:', intercept)

In [ ]:
# making new predictions using our model
predicted_y_value = beta * 0.17 + intercept 
print(f'Given x=0.17 , our model predicts the corresponing y value shoud be {predicted_y_value}')

In [ ]:
plt.scatter(X, y, label='Data Points')
plt.plot(X, beta * X + intercept, color='red', label='Regression Line')
plt.scatter(0.17,predicted_y_value, color='green', label='pred for x = 0.17 ')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression')
plt.legend()
plt.grid(True)
plt.show()


print(f"Calculated beta: {beta}")
print(f"Calculated intercept: {intercept}")


# Calculating the accuracy of our linear regression model




In [ ]:
y_pred = beta * X + intercept

mse = np.mean((y - y_pred)**2)
y_mean = np.mean(y)
r_squared = 1 - np.sum((y - y_pred)**2) / np.sum((y - y_mean)**2)


print("Mean Squared Error (MSE):", mse)
print("R-squared (R^2):", r_squared)



In [ ]:
import json
# read in ./input_json
data = json.load(open("input.json", 'r'))

# convert to numpy arrays
X = np.array(data['input_data'])

### Generate the x and y values in cairo and importing the neccessary libs



In [ ]:
tensor_name =['X_values']

base_path = os.path.join("../../src")

def generate_cairo_files(data, name):
    generated_path = os.path.join(base_path, 'generated')
    os.makedirs(generated_path, exist_ok=True)

    with open(os.path.join(generated_path, f"{name}.cairo"), "w") as f:
            f.write(
                "use array::ArrayTrait;\n" +
                "use orion::operators::tensor::{FP16x16Tensor, TensorTrait, Tensor};\n" +
                "use orion::numbers::{{FixedTrait, FP16x16, FP16x16Impl}};\n"
                "\nfn {0}() -> Tensor<FP16x16>  ".format(name) + "{\n" +
                "    let mut shape = ArrayTrait::new();\n"
            )
            for dim in data.shape:
                f.write("    shape.append({0});\n".format(dim))
            f.write(
                "    let mut data = ArrayTrait::new();\n"
            )
            for val in np.nditer(data.flatten()):
                f.write("    data.append(FixedTrait::new({0}, {1} ));\n".format(abs(int(val * 2**16)), str(val < 0).lower()))
            f.write(
                "let tensor = TensorTrait::<FP16x16>::new(shape.span(), data.span()); \n \n" +
                "return tensor;\n\n"+
                "}\n"
            )
    with open(os.path.join(base_path, 'generated.cairo'), 'w') as f:
        for param_name in tensor_name:
            f.write(f"mod {param_name};\n")

In [ ]:
generate_cairo_files(X, 'X_values')

## Building our OLS functions in cairo using Orion lib

In [ ]:
! touch ../../src/lin_reg_func.cairo

In [ ]:
%%writefile ../../src/lin_reg_func.cairo

use orion::operators::tensor::{Tensor, TensorTrait, FP16x16Tensor};
use orion::numbers::{FP16x16, FixedTrait};
use giza::generated::X_values::X_values;


/// Predicts the y values using the provided x values and computed beta and intercept.
fn main() -> Tensor<FP16x16> {

    // Fetching the x
    let x_values = X_values();
    // precompute the betas (same betas used for ezkl and cairo)
    let mut data = ArrayTrait::new();
    data.append(FixedTrait::new(1, false ));
    data.append(FixedTrait::new(2, false ));
    let betas = TensorTrait::<FP16x16>::new(shape: array![1, 2].span(), data: data.span());
    let intercept = FixedTrait::new(3, false);

    // create a tensor to hold all the y_pred values
    let mut y_pred_shape = array::ArrayTrait::new();
    y_pred_shape.append(x_values.data.len());

    let mut y_pred_vals = array::ArrayTrait::new();

    let mut i: u32 = 0;
    loop {
        if i >=x_values.data.len() {
            break ();
        }
        // (*x_values.data.at(i)).print();
        // TODO figure out how to elementwise multiply a tensor here using Orion.
        // If not add another for loop to do the elementwise multiplication.
        let mut predicted_value = FixedTrait::new(0, true);
        let mut j: u32 = 0;
        loop {
            if j >= betas.data.len() {
                break ();
            }
            predicted_value = *betas.data.at(j) * *x_values.data.at(j) + predicted_value;
            j += 1;
        };
       
        y_pred_vals.append(predicted_value + intercept);
        i += 1;
    };

    let y_pred_tensor = TensorTrait::<FP16x16>::new(y_pred_shape.span(), y_pred_vals.span());

    return y_pred_tensor;
}

## Running tests on our model

In [ ]:
%%writefile ../../src/lib.cairo

mod generated;
mod lin_reg_func;

Build the program

In [ ]:
! scarb build

### Run test

In [ ]:
import subprocess
import time
import json
import os

def run_command_with_gnu_time(cmd, time_cmd):
    full_cmd = f'{time_cmd} -f "%M" -- ' + ' '.join(cmd)
    start_time = time.perf_counter()
    result = subprocess.run(['bash', '-c', full_cmd], capture_output=True, text=True)
    end_time = time.perf_counter()
    print("Full Output:")
    print(result.stdout)
    print(result.stderr)
    # Get the proving time
    proving_time = end_time - start_time
    if result.returncode != 0:
        return (None, None)
    return (result.stderr.strip(), proving_time)  # Memory usage is in stderr

cmd = ["scarb", "cairo-run", "--no-build", "--available-gas 99999999999999999"]

# Get the OS from the environment variable
os_env = os.environ.get('OS')
# gnu time is different on mac and linux, so we try both to remain os agnostic (falls back to mac)
time_command = "/usr/bin/time" if os_env == 'linux' else "gtime"
memory_usage, proving_time = run_command_with_gnu_time(cmd, time_command)

# If memory usage is captured successfully
if memory_usage:
    memory_usage = f"{memory_usage}kb"
else:
    memory_usage = "Memory usage not captured"

# Rest of your process execution and timing code...
    

print(f"Compilation Time: {proving_time} seconds")
print(f"Memory Usage: {memory_usage}")

# define the path that stores the benchmarking results
benchmark_path = os.path.join('../../benchmarks.json')

with open(benchmark_path, 'r') as f:
    benchmark = json.load(f)

proving_time = str(proving_time) + "s"

# Update the proving time in the loaded benchmark
benchmark['linear_regressions']['orion']['provingTime'].append(proving_time)
benchmark['linear_regressions']['orion']['memoryUsage'].append(memory_usage)


# Write the updated benchmark back to the file
with open(benchmark_path, 'w') as f:
    json.dump(benchmark, f, indent=4)